# Importing libraries

In [ ]:
import os, glob, csv, subprocess, sys, re, operator
from git import *
from subprocess import Popen, PIPE
from os import path
import pandas as pd

# Configure repository and directories

In [ ]:
userhome = os.path.expanduser('~')
txt_file = open(userhome + r"/DifferentDiffAlgorithms/SZZ/code_document/project_identity.txt", "r")
pid = txt_file.read().split('\n')
project = pid[0]
bugidentifier = pid[1]
repository = userhome + r'/DifferentDiffAlgorithms/SZZ/datasource/' + project + '/'
analyze_dir = userhome + r'/DifferentDiffAlgorithms/SZZ/projects_analyses/' + project + '/'

print ("Project name = %s" % project)
print ("Project key = %s" % bugidentifier)

# Defining function to execute git command

In [ ]:
def execute_command(cmd, work_dir):
   #Executes a shell command in a subprocess, waiting until it has completed.
    pipe = subprocess.Popen(cmd, shell=True, cwd=work_dir, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    (out, error) = pipe.communicate()
    return out, error
    pipe.wait()

# Loading the files data 

In [ ]:
fields = ['bug_id','bugfix_commitID','parent_id','status','filename']

In [ ]:
dtfiles = pd.read_csv(analyze_dir + "02_diff_extraction/01_modified_files/allmodified_files.csv")
dtfiles = dtfiles[fields]
dtfiles

# Extract the frequency of deleted lines

In [ ]:
algorithms = ['myers','histogram']

In [ ]:
buggylinefiles = []
for n in range(0,len(algorithms)):
    bf = []
    for aa in range(0,len(dtfiles)):
        sys.stdout.write('\r%i: ' %(n+1) + ' Extracting data: %i' %(aa+1))
        sys.stdout.flush()
        
        del_count = "git diff -w --ignore-blank-lines --diff-algorithm=" + algorithms[n] + " " + dtfiles.iloc[aa][2] + " " + dtfiles.iloc[aa][1] + " -- " + dtfiles.iloc[aa][4] + " | grep '^[-]' | grep -Ev '^(--- a/|\+\+\+ b/)' | wc -l"
        del_num = re.search("(\d+)",str(execute_command(del_count, repository)))
        del_num = int(del_num.group())

        zzz = [dtfiles.iloc[aa][0], dtfiles.iloc[aa][1], dtfiles.iloc[aa][2], dtfiles.iloc[aa][4], del_num]
        bf.append(zzz)
        
    buggylinefiles.append(bf)
print('\nExtraction is complete')

In [ ]:
head = ['bug_id','bugfix_commitID','parent_id','filepath','#deletions']
for nn, algo in enumerate(algorithms):
    with open (analyze_dir + '02_diff_extraction/02_list_of_file_with_number_of_deletedlines/' + algo + '_buggylines_files.csv','w') as csvfile:
        writers = csv.writer(csvfile, delimiter=',')
        writers.writerow(head)
        for item in buggylinefiles[nn]:
            writers.writerow(item)
print ("The csv file has been created")

# Merge 2 datasets of modified files that have deleted lines

In [ ]:
diffmyers = pd.read_csv(analyze_dir + '02_diff_extraction/02_list_of_file_with_number_of_deletedlines/myers_buggylines_files.csv')
diffmyers = diffmyers[head][diffmyers[head]['#deletions'] != 0]

diffhist = pd.read_csv(analyze_dir + '02_diff_extraction/02_list_of_file_with_number_of_deletedlines/histogram_buggylines_files.csv')
diffhist = diffhist[head][diffhist[head]['#deletions'] != 0]

In [ ]:
df_merge = diffmyers.merge(diffhist, on=['bug_id','bugfix_commitID','parent_id','filepath'], 
                                        how='outer', suffixes=('_myers', '_histogram'))
df_merge.fillna(0, inplace=True)

In [ ]:
cols = ['bug_id','bugfix_commitID','parent_id','filepath','#deletions_myers','#deletions_histogram']
df_merge = df_merge[cols]
df_merge.drop_duplicates(subset=None, inplace=True)
df_merge

# Deleted lines extraction

In [ ]:
del_files = []
for aa in range(0,len(df_merge)):
    sys.stdout.write('\rExtracting data: %i' %(aa+1))
    sys.stdout.flush()

    f_names = "_" + ((df_merge.iloc[aa][3].split('/'))[-1:])[0] + "_"+ df_merge.iloc[aa][1] + "-" + df_merge.iloc[aa][2][:10] + "_" + df_merge.iloc[aa][0] + "_" + str(aa+1)

    delmyers_num = df_merge.iloc[aa][4]
    if delmyers_num != 0:
        m_name = f_names + "_myersbuglines_" + str(aa+1) + ".diff"
        myers_name = analyze_dir + "02_diff_extraction/03_file_having_deletedlines/myers/" + m_name
        diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=myers " + df_merge.iloc[aa][2] + " " + df_merge.iloc[aa][1] + " -- " + df_merge.iloc[aa][3] + " | grep '^[-]' | grep -Ev '^(--- a/|\+\+\+ b/)' > " + myers_name
        execute_command(diff_cmd, repository)
    else:
        m_name = "-"
        
    delhist_num = df_merge.iloc[aa][5]
    if delhist_num != 0:
        h_name = f_names + "_histogrambuglines_" + str(aa+1) + ".diff"
        hist_name = analyze_dir + "02_diff_extraction/03_file_having_deletedlines/histogram/" + h_name 
        diff_cmd = "git diff -w --ignore-blank-lines --diff-algorithm=histogram " + df_merge.iloc[aa][2] + " " + df_merge.iloc[aa][1] + " -- " + df_merge.iloc[aa][3] + " | grep '^[-]' | grep -Ev '^(--- a/|\+\+\+ b/)' > " + hist_name
        execute_command(diff_cmd, repository)
    else:
        h_name = "-"

    zzz = [df_merge.iloc[aa][0], df_merge.iloc[aa][1], df_merge.iloc[aa][2], df_merge.iloc[aa][3], m_name, h_name, df_merge.iloc[aa][4], df_merge.iloc[aa][5]]
    del_files.append(zzz)
print('\nExtraction is complete')

In [ ]:
header = ['bug_id','bugfix_commitID','parent_id','filepath','filename_myers','filename_histogram','#deletions_myers','#deletions_histogram']
with open (analyze_dir + '02_diff_extraction/03_file_having_deletedlines/buggyfiles_from_both_algorithms.csv','w') as csvfile:
    writers = csv.writer(csvfile, delimiter=',')
    writers.writerow(header)
    for item in del_files:
        writers.writerow(item)
print ("The csv file has been created")